# Analyse de réseaux : les économistes de DBPedia



In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

In [5]:
###  Installation du widget jupyter en plus de la librairie
# jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-cytoscape
import ipycytoscape as cy

In [6]:
import pprint
import csv
import ast
import re

from collections import Counter, defaultdict
from operator import itemgetter

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

import math

## Base de données

In [9]:
### Définir les adresses des fichiers, l'existant et celui à créer
db = 'data/sparql_queries.db'

In [ ]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

In [ ]:
##  Noter que la différence de suffixe, en soi totalement arbitraire, 
#  dépend du fait que dans la configuration de .gitignore, .sqlite est exclu du verisonnement GIT
#  contrairement à .db qui est versionné
timestamped_db_copy = 'data/sparql_queries_' + timestamp_formatted_for_file_name() + '.sqlite'

In [ ]:
### Créer une copie de sauvegarde avec timestamp du fichier (versionnement manuel)
# ATTENTION : la base de données doit exister à l'endroit indiqué !
## Cette requête n'est utile que si des modifications en écriture vont être apportées à la base de données,
# afint de préserver le dernier état avant modification

## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(db, timestamped_db_copy)
copied_db

## Relations pendant les études

Personnes ayant étudié dans la même université dans une fourchette de 12 ans : on émet l'hypothèse que ils/elles ont virtuellement pu nouer des relations qui peuvent influences les parcours et choix ultérieurs grâce à des contacts prolongés avec d'anciens camarades d'études

In [ ]:
### 
#  On part de la date de naissance et on postule un croisement entre une et plusieurs années
# en admettant que l'âge des études soit approximativement le même

etudes_meme_uni = """
SELECT REPLACE(p.uri_entity , 'http://dbpedia.org/resource/', '') person_1,
REPLACE(p1.uri_entity , 'http://dbpedia.org/resource/', '') person_2,
REPLACE(p.value, 'http://dbpedia.org/resource/', '') alma_mater,
p2.value birth_year_1,
p3.value birth_year_2,
p4.value long,
p5.value lat
FROM property p
  JOIN property p1 ON p1.value = p.value AND p1.pk_property > p.pk_property AND p1.property LIKE '%almaMater'
  JOIN property p2 ON p2.uri_entity = p.uri_entity AND p2.property LIKE '%irthYe%' AND p2."source" LIKE '%8 mai 2021%'
  JOIN property p3 ON p3.uri_entity = p1.uri_entity AND p3.property LIKE '%irthYe%' AND p3."source" LIKE '%8 mai 2021%'
  LEFT JOIN property p4 ON
	p4.uri_entity = p1.value
	AND p4.property LIKE '%long'
LEFT JOIN property p5 ON
	p5.uri_entity = p1.value
	AND p5.property LIKE '%lat'
WHERE  p.property LIKE '%almaMater'
AND p3.value BETWEEN (p2.value - 6) AND (p2.value + 6);
"""

In [ ]:
### Undirected version of the same graph produced using Pandas
cn = sql.connect(db)
same_univ = pd.read_sql_query(etudes_meme_uni, cn)
cn.close()
same_univ.head()

In [ ]:
universities = same_univ.groupby(['alma_mater']).size()
print(len(universities))
list(universities.sort_values(ascending=False).items())[:100] #[10:20]

In [ ]:
G = nx.from_pandas_edgelist(same_univ, 'person_1', 'person_2', edge_attr='alma_mater')
print(nx.info(G))

In [ ]:
G.is_multigraph(), G.is_directed(), G.number_of_nodes(), G.number_of_edges(), nx.is_connected(G), nx.density(G)

In [ ]:
nx.number_connected_components(G)

In [ ]:
## Centralité et dispersion du degré
df_degree = pd.DataFrame(list(nx.degree(G)), columns = ['person', 'degree'])
df_degree.head()
df_degree.set_index('person',inplace=True)
df_degree.head()

In [ ]:
df_degree['degree'].describe()

In [ ]:
gr_degree = df_degree.groupby(['degree']).size()
gr_degree.sort_index()

In [ ]:
### Distribution des naissances dans le temps. Pour mémoire : 2000 = sans année de naissance


gr_degree = df_degree.groupby(['degree']).size()


objects = [l for l in gr_degree.index]
eff = [l for l in gr_degree]

print(objects[:5])
plt.figure(figsize=(15,7))
plt.bar(objects, eff)

In [ ]:
df_degree[df_degree['degree'] > 80]

In [ ]:
nx.number_connected_components(G)

In [ ]:
### Composantes du graphe
# Une composante principale avec 1402 individus et de multiples petites composantes
# https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html
c_comp = nx.connected_components(G)
print(type(c_comp))
## Length of generator as set of nodes = number of nodes
df_c_comp = pd.DataFrame([len(c) for c in sorted(c_comp, key=len, reverse=True)], columns=['eff'])
gs_c_comp = df_c_comp.groupby(['eff']).size()
gs_c_comp.sort_index()

In [ ]:
### Créer les sous-graphes correspondant aux composantes
#  https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html
S = [[len(c), G.subgraph(c).copy()] for c in nx.connected_components(G)]
# type(S)

In [ ]:
### Choisir la/les composante(s) à 9 noeuds 
a = [[s[0], s[1]] for s in S  if s[0] == 8]
a

In [ ]:
sub_g = a[0][1]
ll = [{(l[0],l[1]):l[2]['alma_mater']} for l in list(sub_g.edges.data())]
edge_labels = defaultdict(list)
{edge_labels[key].append(sub[key]) for sub in ll for key in sub} 

In [ ]:

plt.figure(figsize=(20,10))
pos = nx.circular_layout(sub_g)
el = nx.draw_networkx_edge_labels(sub_g, pos, edge_labels = edge_labels, font_size = 10, font_color = 'red')
# print(edge_labels)
nx.draw_networkx(sub_g,pos)


In [ ]:
### Choisir la/les composante(s) à 9 noeuds 
a7 = [[s[0], s[1]] for s in S  if s[0] == 7]
a7

In [ ]:
for g in a7:
    sub_g = g[1]
    ll = [{(l[0],l[1]):l[2]['alma_mater']} for l in list(sub_g.edges.data())]
    edge_labels = defaultdict(list)
    e = {edge_labels[key].append(sub[key]) for sub in ll for key in sub} 
    plt.figure(figsize=(20,10))
    pos = nx.circular_layout(sub_g)
    el = nx.draw_networkx_edge_labels(sub_g, pos, edge_labels = edge_labels, font_size = 10, font_color = 'red')
    # print(edge_labels)
    nx.draw_networkx(sub_g,pos)

## Sous-graphe

In [ ]:
### Choisir la/les composante(s) à 1402 noeuds 
a1402= [[s[0], s[1]] for s in S  if s[0] == 1402]
print(a1402)
sub_g = a1402[0][1]

In [ ]:
print(nx.info(sub_g))

In [ ]:
nx.is_connected(sub_g), nx.density(sub_g)

In [ ]:
### Returns all maximal cliques in an undirected graph.
# https://networkx.org/documentation/stable/reference/algorithms/clique.html?highlight=cliques

cliques = [(len(c),c) for c in list(nx.find_cliques(sub_g)) if len(c) > 2]
print(len(cliques))
cliques[-3:]

In [ ]:
df_cliques = pd.DataFrame([l[0] for l in cliques], columns = ['eff'])
gs_cliques = df_cliques.groupby(['eff']).size()
gs_cliques.sort_index()

In [ ]:
[c for c in cliques if c[0] == 32]

In [ ]:
pos = ''
cc = [c[1] for c in cliques if c[0] == 32]

sub_32 = G.subgraph(cc[0]).copy()
ll = [{(l[0],l[1]):l[2]['alma_mater']} for l in list(sub_32.edges.data())]
edge_labels = defaultdict(list)
e = {edge_labels[key].append(sub[key]) for sub in ll for key in sub} 
plt.figure(figsize=(20,10))
pos = nx.spring_layout(sub_32)
el = nx.draw_networkx_edge_labels(sub_32, pos, edge_labels = edge_labels, font_size = 7, alpha = 0.7, font_color = 'green')
# print(edge_labels)
nx.draw_networkx(sub_32,pos)

In [ ]:
### https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.connectivity.kcomponents.k_components.html#networkx.algorithms.connectivity.kcomponents.k_components

##  exécution très longue : vérifier
# nx.k_components(sub_g)

In [ ]:
###https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.approximation.kcomponents.k_components.html?highlight=k_components#networkx.algorithms.approximation.kcomponents.k_components

from networkx.algorithms import approximation as apxa
##  exécution très longue : vérifier
# apxa.k_components(sub_g, min_density = 0.95)

## Sous-graphe : degré > 13

In [ ]:
lk_13 = list(df_degree['person'][df_degree['degree']> 13])
lk_13[:3]

In [ ]:
#  https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html
dk_13 = G.subgraph(lk_13).copy()
print(nx.info(dk_13))

In [ ]:
nx.number_connected_components(dk_13)

In [ ]:
### Composantes du graphe
# Une composante principale avec 1402 individus et de multiples petites composantes
# https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html
c_comp = nx.connected_components(dk_13)
print(type(c_comp))
## Length of generator as set of nodes = number of nodes
df_c_comp = pd.DataFrame([len(c) for c in sorted(c_comp, key=len, reverse=True)], columns=['eff'])
gs_c_comp = df_c_comp.groupby(['eff']).size()
gs_c_comp.sort_index()

In [ ]:
#  https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html
S = [[len(c), G.subgraph(c).copy()] for c in nx.connected_components(dk_13)]
# type(S)

In [ ]:
a = [[s[0], s[1]] for s in S  if s[0] == 542]
g542 = a[0][1]

In [ ]:
fig = plt.figure(figsize=(400,200))
pos = nx.kamada_kawai_layout(g542)
nx.draw_networkx(g542, pos , font_color = 'red', font_size=30, edge_color='LightGray' )
plt.savefig("graphs/dk_13.jpg", format="jpg")
plt.savefig("graphs/dk_13.pdf", format="pdf")
plt.show()

In [ ]:
node_sizes = [i[1]['betweenness'] * 100000 + 1000 for i in list(g542.nodes.data())]
print(len(node_sizes))
node_sizes[:4]

In [ ]:
fig = plt.figure(figsize=(200,100))
pos = nx.kamada_kawai_layout(g542)
nx.draw_networkx(g542, pos, node_color = 'DarkBlue', font_color = 'CornflowerBlue',\
                 font_size=30, node_size=node_sizes, edge_color='LightGray' )
plt.savefig("graphs/g542.pdf", format="pdf")
plt.savefig("graphs/g542.jpg", format="jpg")
plt.show()

In [ ]:
node_sizes = [i[1]['eigenvector'] * 100000 + 1000 for i in list(g542.nodes.data())]
print(len(node_sizes))
node_sizes[:4]

In [ ]:
fig = plt.figure(figsize=(200,100))
pos = nx.kamada_kawai_layout(g542)
nx.draw_networkx(g542, pos, node_color = 'DarkBlue', font_color = 'CornflowerBlue',\
                 font_size=30, node_size=node_sizes, edge_color='LightGray' )
plt.savefig("graphs/g542_eigenvector.pdf", format="pdf")
plt.savefig("graphs/g542_eigenvector.jpg", format="jpg")
plt.show()

## Ajouter les paramètres de centralité du sous-graphe g542



### Centralité de degré

In [ ]:
# On construit ici le dictionnaire selon le format souhaité
degree = dict([(d[0], {'degree': d[1]}) for d in nx.degree(g542)])
print(list(degree)[:3])

In [ ]:
nx.set_node_attributes(g542, degree)

### Centralité de proximité

In [ ]:
closeness = nx.closeness_centrality(g542)

In [ ]:
nx.set_node_attributes(g542, closeness, 'closeness')

### Centralité d'intermédiarité

In [ ]:
betweenness = nx.betweenness_centrality(g542)

In [ ]:
nx.set_node_attributes(g542, betweenness, 'betweenness')

### Centralité d'influence ou de pouvoir (_eigenvalue_)

In [ ]:
### https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.eigenvector_centrality.html?highlight=eigenvector#networkx.algorithms.centrality.eigenvector_centrality
# noter la référence biliographique dans la documentation de la fonction

eigenvector = nx.eigenvector_centrality(g542)
nx.set_node_attributes(g542, eigenvector, 'eigenvector')

## Résultat

In [ ]:
print(list(g542.nodes.data())[:5])

## Exporter les attributs pour les explorer 

In [ ]:
### Exporter les attributs vers un DataFrame afin de les explorer
export = pd.DataFrame(g542.nodes.data(), columns = ['id', 'attributes'])
export

In [ ]:
### Décomposer le champs attributs en colonnes
# https://stackoverflow.com/questions/38231591/split-explode-a-column-of-dictionaries-into-separate-columns-with-pandas
pd.json_normalize(export.attributes)

In [ ]:
### appliquer au DataFrame et supprimer la colonne attributes
# en même temps mettre le nom en index
export = export.join(pd.json_normalize(export.attributes)).set_index('id')
export.drop(columns=['attributes'], inplace=True)

In [ ]:
export.loc[['Klaus_Schwab']]

In [ ]:
export['closeness'].describe()

In [ ]:
export['betweenness'].describe()

In [ ]:
export['eigenvector'].describe()

In [ ]:
export.sort_values(by=['eigenvector'], ascending=False).iloc[50:90]  # .iloc[50:90]   .head(50)

In [ ]:
export.loc[['Klaus_Schwab']]

In [ ]:
export.loc[export.index.isin(['Klaus_Schwab'])]  ### négation :  loc[~export.index.isin(['Ellen_Johnson_Sirleaf'])]  

## Explorer les relations entre propriétés structurales (centralités) et/ou attributs

### Relations entre deux centralités

In [ ]:
export[['degree','betweenness']].plot(x = 'degree', y = 'betweenness',kind='scatter',figsize=(30,30))
for i,row in list(export[['degree','betweenness']].iterrows()):
    plt.annotate(i,(row['degree'], row['betweenness']), fontsize = 9, stretch='semi-expanded')

In [ ]:
export.sort_values(by=['eigenvector'], ascending=False).iloc[80:100]

In [ ]:
len(export), len(export.loc[~export.index.isin(['Ellen_Johnson_Sirleaf', 'Fischer_Black', 'Shahid_Javed_Burki'])])

In [ ]:
exp_Schwab = export[['closeness','eigenvector']].loc[~export.index.isin(['Ellen_Johnson_Sirleaf', 'Fischer_Black', 'Shahid_Javed_Burki'])][export['eigenvector']> 0.05]\
.sort_values('eigenvector', ascending=False).copy(deep=True)

In [ ]:
exp_Schwab.sort_values('eigenvector', ascending=False).plot(x = 'closeness', y = 'eigenvector',kind='scatter',figsize=(30,40))
for i,row in list(exp_Schwab.iterrows()):
    plt.annotate(i,(row['closeness'], row['eigenvector']), fontsize = 10, stretch='expanded')
plt.savefig('plots/closeness_eigenvector_20210516.jpg')

## Ego réseau

In [ ]:
schwab_ego = nx.ego_graph(g542, 'Klaus_Schwab', radius = 2, undirected=True)
print(len(schwab_ego))

In [ ]:
schwab_ego

In [ ]:
node_sizes = [i[1]['betweenness'] * 100000 + 1000 for i in list(schwab_ego.nodes.data())]
print(len(node_sizes))
node_sizes[:4]

In [ ]:
fig = plt.figure(figsize=(200,100))
pos = nx.kamada_kawai_layout(schwab_ego)
nx.draw_networkx(schwab_ego, pos, node_color = 'DarkBlue', font_color = 'CornflowerBlue',\
                 font_size=30, node_size=node_sizes, edge_color='LightGray' )
plt.savefig("graphs/schwab_ego.pdf", format="pdf")
plt.show()

# Liens entre les Organisations (en préparation)

In [45]:
### Définir les adresses des fichiers, l'existant et celui à créer
db = 'data/sparql_queries.db'

In [48]:
## liens entre Universités donnés par les étudiants
liens_univ = """
WITH tw1 AS (
SELECT REPLACE(p.value , 'http://dbpedia.org/resource/', '') orga_1,
REPLACE(p1.value, 'http://dbpedia.org/resource/', '') orga_2,
'les_deux_alma_mater' AS type_relation,
REPLACE(p.uri_entity, 'http://dbpedia.org/resource/', '') person,
p3.value birth_year,
p4.value long,
p5.value lat,
p6.value long,
p7.value lat
FROM property p
  JOIN property p1 ON p1.uri_entity = p.uri_entity AND p1.pk_property > p.pk_property AND p1.property LIKE '%almaMater'
  JOIN property p3 ON p3.uri_entity = p1.uri_entity AND p3.property LIKE '%irthYe%' AND p3."source" LIKE '%8 mai 2021%'
  LEFT JOIN property p4 ON
	p4.uri_entity = p.value
	AND p4.property LIKE '%long'
	LEFT JOIN property p5 ON
	p5.uri_entity = p.value
	AND p5.property LIKE '%lat'
	LEFT JOIN property p6 ON
	p6.uri_entity = p1.value
	AND p6.property LIKE '%long'
LEFT JOIN property p7 ON
	p7.uri_entity = p1.value
	AND p7.property LIKE '%lat'
WHERE  p.property LIKE '%almaMater'
UNION
SELECT REPLACE(p.value , 'http://dbpedia.org/resource/', '') orga_1,
REPLACE(p1.value, 'http://dbpedia.org/resource/', '') orga_2,
'les_deux_organisation' AS type_relation,
REPLACE(p.uri_entity, 'http://dbpedia.org/resource/', '') person,
p3.value birth_year,
p4.value long,
p5.value lat,
p6.value long,
p7.value lat
FROM property p
  JOIN property p1 ON p1.uri_entity = p.uri_entity AND p1.pk_property > p.pk_property AND p1.property LIKE '%instit%'
  JOIN property p3 ON p3.uri_entity = p1.uri_entity AND p3.property LIKE '%irthYe%' AND p3."source" LIKE '%8 mai 2021%'
  LEFT JOIN property p4 ON
	p4.uri_entity = p.value
	AND p4.property LIKE '%long'
	LEFT JOIN property p5 ON
	p5.uri_entity = p.value
	AND p5.property LIKE '%lat'
	LEFT JOIN property p6 ON
	p6.uri_entity = p1.value
	AND p6.property LIKE '%long'
LEFT JOIN property p7 ON
	p7.uri_entity = p1.value
	AND p7.property LIKE '%lat'
WHERE  p.property LIKE '%instit%'
UNION
SELECT REPLACE(p.value , 'http://dbpedia.org/resource/', '') orga_1,
REPLACE(p1.value, 'http://dbpedia.org/resource/', '') orga_2,
'alma_mater_vers_organisation' AS type_relation,
REPLACE(p.uri_entity, 'http://dbpedia.org/resource/', '') person,
p3.value birth_year,
p4.value long,
p5.value lat,
p6.value long,
p7.value lat
FROM property p
  JOIN property p1 ON p1.uri_entity = p.uri_entity  AND p1.property LIKE '%instit%' -- AND p1.pk_property > p.pk_property
  JOIN property p3 ON p3.uri_entity = p1.uri_entity AND p3.property LIKE '%irthYe%' AND p3."source" LIKE '%8 mai 2021%'
  LEFT JOIN property p4 ON
	p4.uri_entity = p.value
	AND p4.property LIKE '%long'
	LEFT JOIN property p5 ON
	p5.uri_entity = p.value
	AND p5.property LIKE '%lat'
	LEFT JOIN property p6 ON
	p6.uri_entity = p1.value
	AND p6.property LIKE '%long'
LEFT JOIN property p7 ON
	p7.uri_entity = p1.value
	AND p7.property LIKE '%lat'
WHERE  p.property LIKE '%almaMater%')
SELECT *  FROM tw1
WHERE orga_1 NOT LIKE 'Bachel%' 
AND orga_2 NOT LIKE 'Bachel%'
AND orga_1 NOT LIKE 'Master%' 
AND orga_2 NOT LIKE 'Master%'
AND orga_1 NOT LIKE 'Doctor%' 
AND orga_2 NOT LIKE 'Doctor%'
ORDER BY person;
"""

In [91]:
### Undirected version of the same graph produced using Pandas
cn = sql.connect(db)
df_liens_univ = pd.read_sql_query(liens_univ, cn)
cn.close()
df_liens_univ.head()

,orga_1,orga_2,type_relation,person,birth_year,long,lat,long:1,lat:1
0,Bangalore_University,Harvard_University,les_deux_alma_mater,A._K._Shiva_Kumar,1956,77.5032,12.9388,-71.1169,42.3744
1,Bangalore_University,IIM_Ahmedabad,les_deux_alma_mater,A._K._Shiva_Kumar,1956,77.5032,12.9388,None,None
2,Bangalore_University,"St._Joseph's_College,_Bangalore",les_deux_alma_mater,A._K._Shiva_Kumar,1956,77.5032,12.9388,77.5964,12.9629
3,IIM_Ahmedabad,Harvard_University,les_deux_alma_mater,A._K._Shiva_Kumar,1956,None,None,-71.1169,42.3744
4,IIM_Ahmedabad,"St._Joseph's_College,_Bangalore",les_deux_alma_mater,A._K._Shiva_Kumar,1956,None,None,77.5964,12.9629


In [ ]:
students = df_liens_univ.groupby(['person']).size()
print(len(students))
list(students.sort_values(ascending=False).items())[:10] #[10:20]

In [90]:
## Piet_Lieftinck 8 universities but errors 
df_liens_univ[df_liens_univ['person'] == 'Milton_Friedman']

,orga_1,orga_2,type_relation,person,birth_year,long,lat,long:1,lat:1
4211,Columbia_University,Columbia_University,alma_mater_vers_organisation,Milton_Friedman,1912,-73.9619,40.8075,-73.9619,40.8075
4212,Columbia_University,Hoover_Institution,alma_mater_vers_organisation,Milton_Friedman,1912,-73.9619,40.8075,None,None
4213,Columbia_University,U.S._Department_of_the_Treasury,alma_mater_vers_organisation,Milton_Friedman,1912,-73.9619,40.8075,None,None
4214,Columbia_University,U.S._Department_of_the_Treasury,les_deux_organisation,Milton_Friedman,1912,-73.9619,40.8075,None,None
4215,Columbia_University,University_of_Cambridge,alma_mater_vers_organisation,Milton_Friedman,1912,-73.9619,40.8075,0.117222,52.2053
4216,Columbia_University,University_of_Chicago,alma_mater_vers_organisation,Milton_Friedman,1912,-73.9619,40.8075,-87.5997,41.7897
4217,Columbia_University,"University_of_Wisconsin,_Madison",alma_mater_vers_organisation,Milton_Friedman,1912,-73.9619,40.8075,None,None
4218,Hoover_Institution,Columbia_University,les_deux_organisation,Milton_Friedman,1912,None,None,-73.9619,40.8075
4219,Hoover_Institution,U.S._Department_of_the_Treasury,les_deux_organisation,Milton_Friedman,1912,None,None,None,None
4220,Rutgers_University,Columbia_University,alma_mater_vers_organisation,Milton_Friedman,1912,-74.4481,40.5017,-73.9619,40.8075


In [ ]:
df_students = pd.DataFrame(students, columns=['eff'])
gdf_students = df_students.groupby(['eff']).size()
gdf_students.sort_index(ascending=True)[:20]

In [92]:
GU = nx.from_pandas_edgelist(df_liens_univ, 'orga_1', 'orga_2')
type(GU)

networkx.classes.graph.Graph

In [93]:
### CA DOIT être un multigraphe !

In [94]:
GU.is_multigraph(), GU.is_directed(), GU.number_of_nodes(), GU.number_of_edges(), nx.is_connected(GU), nx.density(GU)

(False, False, 1462, 4812, False, 0.004505655946538875)

In [95]:
nx.number_connected_components(GU)

71

In [96]:
### Composantes du graphe
# Une composante principale avec 1505 individus et de multiples petites composantes
# https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html
c_comp = nx.connected_components(GU)
print(type(c_comp))
## Length of generator as set of nodes = number of nodes
df_c_comp = pd.DataFrame([len(c) for c in sorted(c_comp, key=len, reverse=True)], columns=['eff'])
gs_c_comp = df_c_comp.groupby(['eff']).size()
gs_c_comp.sort_index()

<class 'generator'>


eff
1        3
2       52
3       12
4        3
1307     1
dtype: int64

In [98]:
#  https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html
#  print([c for c in c_comp if len(c) == 4])
S = [[len(c), GU.subgraph(c).copy()] for c in nx.connected_components(GU)]
# type(S)

In [101]:
S[0]

[1307, <networkx.classes.graph.Graph at 0x15cf79610>]

In [ ]:
plt.figure(figsize=(15,7))
pos = nx.kamada_kawai_layout(S[0][1])
nx.draw_networkx(GU, pos, node_color = 'DarkBlue', font_color = 'CornflowerBlue',\
                 font_size=30, node_size=node_sizes, edge_color='LightGray' )
nx.draw_networkx(S[0][1])


In [ ]:
fig = plt.figure(figsize=(200,100))
pos = nx.kamada_kawai_layout(GU)
nx.draw_networkx(GU, pos, node_color = 'DarkBlue', font_color = 'CornflowerBlue',\
                 font_size=30, node_size=node_sizes, edge_color='LightGray' )
plt.savefig("graphs/GU.pdf", format="pdf")
plt.show()

In [80]:
au = [[s[0], s[1]] for s in GU  if s[0] == 4]
len(au)
#a

0

In [ ]:
gg1307 = [[s[0], s[1]] for s in S  if s[0] == 1307]
print(len(gg1307))
len(gg)
gg1307[1]

In [73]:
len(gg1307)

1

In [67]:
gg1307 = gg[0][1]
type(gg1307)

networkx.classes.graph.Graph

In [71]:
gg1307 = [0][0]

### Centralité d'influence ou de pouvoir (_eigenvalue_)

In [41]:
### https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.eigenvector_centrality.html?highlight=eigenvector#networkx.algorithms.centrality.eigenvector_centrality
# noter la référence biliographique dans la documentation de la fonction

eigenvector = nx.eigenvector_centrality(g911)
nx.set_node_attributes(g911, eigenvector, 'eigenvector')

In [ ]:
g911[0][1]

## Résultat

In [ ]:
print(list(g911.nodes.data())[:5])

In [ ]:
eigenvector

## Exporter les attributs pour les explorer 

In [ ]:
### Exporter les attributs vers un DataFrame afin de les explorer
export = pd.DataFrame(g542.nodes.data(), columns = ['id', 'attributes'])
export

In [ ]:
### Décomposer le champs attributs en colonnes
# https://stackoverflow.com/questions/38231591/split-explode-a-column-of-dictionaries-into-separate-columns-with-pandas
pd.json_normalize(export.attributes)

In [ ]:
### appliquer au DataFrame et supprimer la colonne attributes
# en même temps mettre le nom en index
export = export.join(pd.json_normalize(export.attributes)).set_index('id')
export.drop(columns=['attributes'], inplace=True)

In [ ]:
export.loc[['Klaus_Schwab']]

In [ ]:
export['closeness'].describe()

In [ ]:
export['betweenness'].describe()

In [ ]:
export['eigenvector'].describe()

In [ ]:
fig = plt.figure(figsize=(200,100))
pos = nx.kamada_kawai_layout(GU)
nx.draw_networkx(GU, pos, node_color = 'DarkBlue', font_color = 'CornflowerBlue',\
                 font_size=30, node_size=node_sizes, edge_color='LightGray' )
plt.savefig("graphs/GU.pdf", format="pdf")
plt.show()

In [ ]:
influences_df[influences_df['person_1']=='Stati_Statev']

In [ ]:
influences_df[influences_df['alma_mater']=='University_of_National_and_World_Economy']

In [ ]:
a[0][1].nodes().data()

In [ ]:
list(G.nodes().data())[:3]

In [ ]:
### La composante la plus grande
max_c = [[s[0], s[1]] for s in S  if s[0] == max([s[0] for s in S])]
max_c

In [ ]:
plt.figure(figsize=(20,10))
nx.draw_networkx(max_c[0][1])

In [ ]:
nx.number_strongly_connected_components(max_c[0][1])

In [ ]:
[[a, a_dict.items()] for a, a_dict in G.adjacency()][:1]

In [ ]:
G.adjacency()

In [ ]:
list(G.nodes)[:5]

In [ ]:
list(G.neighbors('Milton_Friedman'))

In [ ]:
type(G), type(DG)

In [ ]:
# list(nx.bridges(G))

In [ ]:
degrees = list(nx.degree(G))
degrees[:3]

In [ ]:
df_degrees = pd.DataFrame(degrees, columns=['name', 'degree']).sort_values(by='degree', ascending=False)
df_degrees.head(100)

In [ ]:
gb_degrees = df_degrees.groupby(['degree']).size()
gb_degrees.sort_index() # values(by='degree', ascending=False).head(30)

In [ ]:
### Distribution des naissances dans le temps. Pour mémoire : 2000 = sans année de naissance


objects = [l for l in gb_degrees.index]
eff = [l for l in gb_degrees]

plt.figure(figsize=(15,7))
plt.bar(objects, eff)

In [ ]:
sup4 = [dd[0] for dd in degrees if dd[1] > 4]
len(sup4)

In [ ]:
d_sup_1 = nx.restricted_view(G, [dd[0] for dd in degrees if dd[1]>1], [])
type(d_sup_1)

In [ ]:
d_sup4 = nx.restricted_view(G, sup4, [])
type(d_sup4)

In [ ]:
max_cli_sup4 = nx.make_max_clique_graph(d_sup4)

In [ ]:
type(max_cli_sup4)

In [ ]:
type(nx.k_core(d_sup4, 10))

In [ ]:
list(nx.core_number(d_sup4).items())[:3]

In [ ]:
kcg = nx.k_core(d_sup4)
plt.figure(figsize=(40,20))
nx.draw_networkx(kcg,pos=nx.spring_layout(kcg))

In [ ]:
plt.figure(figsize=(40,20))
nx.draw_networkx(max_cli_sup4,pos=nx.spring_layout(max_cli_sup4))

In [ ]:
list(max_cli_sup4.adjacency())

In [ ]:
plt.figure(figsize=(40,20))
nx.draw_kamada_kawai(d_sup4)

In [ ]:
GG = d_sup_1
pos = nx.spring_layout(GG)

node_sizes = [i for i in range(len(GG))]
M = GG.number_of_edges()
edge_colors = range(2, M + 2)
edge_alphas = [(5 + i) / (M + 4) for i in range(M)]
cmap = plt.cm.plasma

nodes = nx.draw_networkx_nodes(GG, pos, node_size=node_sizes, node_color="indigo")
edges = nx.draw_networkx_edges(
    GG,
    pos,
    node_size=node_sizes,
    arrowstyle="->",
    arrowsize=10,
    edge_color=edge_colors,
    edge_cmap=cmap,
    width=2,
)
# set alpha value for each edge
for i in range(M):
    edges[i].set_alpha(edge_alphas[i])

pc = mpl.collections.PatchCollection(edges, cmap=cmap)
pc.set_array(edge_colors)
plt.colorbar(pc)

ax = plt.gca()
ax.set_axis_off()
plt.show()

In [ ]:


plt.figure(figsize=(20,20))
pos = graphviz_layout(d_sup_1)
nx.draw_networkx_labels(d_sup_1, pos)

In [ ]:
nx.draw(G)

In [ ]:
from networkx.drawing.nx_agraph import graphviz_layout
pos = graphviz_layout(G)
plt.axis('off')
nx.draw_networkx_nodes(G,pos,node_color='g',alpha = 0.8)  # draws nodes
nx.draw_networkx_edges(G,pos,edge_color='b',alpha = 0.6)  # draws edges
nx.draw_networkx_edge_labels(G,pos,edge_labels = nx.get_edge_attributes(G,'weight')) # edge lables
nx.draw_networkx_labels(G,pos) # node lables

## Relations d'influence


Préparation des données (cf. le carnet dbpedia_production):


### Requêtes préliminaires

* Requêtes d'exploration dans la base de données:

<code>
-- nombre de personnes qui influencent : 906
WITH tw1 AS (SELECT DISTINCT uri_entity FROM property p WHERE p.property LIKE '%influenced%')
SELECT count(*) FROM tw1;

-- nombre de personnes influencées : 1173
WITH tw1 AS (SELECT DISTINCT value FROM property p WHERE p.property LIKE '%influenced%')
SELECT count(*) FROM tw1;

-- nombre total de personnes dans le réseau : 1717
WITH tw1 AS (
SELECT DISTINCT uri_entity FROM property p WHERE p.property LIKE '%influenced%'
UNION 
SELECT DISTINCT value as uri_entity FROM property p WHERE p.property LIKE '%influenced%')
SELECT COUNT(*) FROM tw1;


-- economistes (entités déjà disponibles) dans le réseau : 933
-- personnes non encore connues: 784
WITH tw1 AS (
SELECT DISTINCT uri_entity FROM property p WHERE p.property LIKE '%influenced%'
UNION 
SELECT DISTINCT value as uri_entity FROM property p WHERE p.property LIKE '%influenced%')
SELECT COUNT(*) FROM tw1 LEFT JOIN entity e ON tw1.uri_entity = e.uri_entity AND e.entity_class LIKE '%Person%'
WHERE e.uri_entity IS NULL;

</code>


In [ ]:
### Définir les adresses des fichiers, l'existant et celui à créer
db = 'data/sparql_queries.db'

In [ ]:
### Vérifier si tous les lieux on un type
influence ="""
SELECT REPLACE(p.uri_entity, 'http://dbpedia.org/resource/', '') influencer, REPLACE(p.value, 'http://dbpedia.org/resource/', '') influenced  
FROM property p WHERE p.property LIKE '%influenced%'
AND p.uri_entity != p.value;
"""

In [ ]:
# connnexion à la base de données
cn = sql.connect(db)
c = cn.cursor()

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c.execute(influence)
result_q = []
result_q = c.fetchall()

# fermer la connexion
cn.close()
print(len(result_q))
result_q[:4]

In [ ]:
#DiGraph : directed graph
DG = nx.DiGraph()
DG.add_edges_from(result_q, color='blue')
type(DG)

In [ ]:
DG.size()

In [ ]:
### Propriétés du graphe
DG.is_multigraph(), DG.is_directed(), DG.number_of_nodes(), DG.number_of_edges(), \
nx.density(DG), nx.is_strongly_connected(DG), nx.is_weakly_connected(DG)

In [ ]:
nx.number_strongly_connected_components(DG), nx.number_weakly_connected_components(DG)

In [ ]:
### Composantes du graphe (weakly connected = indirected)
# Une composante principale avec 1505 individus et de multiples petites composantes
# https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html
wc_comp = nx.weakly_connected_components(DG)
print(type(wc_comp))
## Length of generator as set of nodes = number of nodes
df_wc_comp = pd.DataFrame([len(c) for c in sorted(wc_comp, key=len, reverse=True)], columns=['eff'])
gs_wc_comp = df_wc_comp.groupby(['eff']).size()
gs_wc_comp.sort_index()

In [ ]:
### Composantes du graphe (strongly connected = directed)
# Une composante principale avec 1505 individus et de multiples petites composantes
# https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html
sc_comp = nx.strongly_connected_components_recursive(DG)
print(type(c_comp))
## Length of generator as set of nodes = number of nodes
df_sc_comp = pd.DataFrame([len(c) for c in sorted(sc_comp, key=len, reverse=True)], columns=['eff'])
gs_sc_comp = df_c_comp.groupby(['eff']).size()
gs_sc_comp.sort_index()

In [ ]:
#  https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html
S = [[len(c), DG.subgraph(c).copy()] for c in nx.weakly_connected_components(DG)]
len(S)

In [ ]:
adg = [[s[0], s[1]] for s in S  if s[0] == 11]
adg

In [ ]:
## https://networkx.org/documentation/stable/reference/drawing.html
plt.figure(figsize=(15,7))

pos = nx.shell_layout(adg[0][1])
#pos = nx.kamada_kawai_layout(a[0][1])

## Attention pas plus d'une cinquantaine de noeuds
nx.draw_networkx(adg[0][1], pos)

In [ ]:
be_ce = nx.betweenness_centrality(DG)

In [ ]:
df_be_ce = pd.DataFrame(list(be_ce.items()), columns=['name', 'be_ce'])
df_be_ce.head(30)

In [ ]:
df_be_ce['be_ce'].sort_values(ascending=False)

In [ ]:
l_de_ce = list(de_ce.items())

In [ ]:
[[l[0],l[1]*1000] for l in l_de_ce][:3]

In [ ]:
dfn = pd.DataFrame([[l[0],l[1]*1000] for l in l_de_ce], columns=['name', 'be_ce'])
dfn.head()

In [ ]:
dfn.sort_values(by = ['be_ce'], ascending=False).head(10)


In [ ]:
cl_ce = nx.closeness_centrality(DG)

In [ ]:
df_cl_ce = pd.DataFrame(list(cl_ce.items()), columns=['name', 'cl_ce'])
df_cl_ce.head(10)

In [ ]:
df_cl_ce['cl_ce'].sort_values(ascending=False)

In [ ]:
l_cl_ce = list(cl_ce.items())

In [ ]:
[[l[0],l[1]*1000] for l in l_cl_ce][:3]

In [ ]:
dfn = pd.DataFrame([[l[0],l[1]*1000] for l in l_de_ce], columns=['name', 'be_ce'])
dfn.head()

In [ ]:
dfn.sort_values(by = ['be_ce'], ascending=False).head(30)


In [ ]:
max([l[1] for l in l_de_ce][:3])

In [ ]:
min([l[1] for l in l_de_ce]), max([l[1] for l in l_de_ce])

In [ ]:
print(max([l[1] for l in l_de_ce][50:60]))
ll = l_de_ce[50:60]
ll

In [ ]:
ll = [l[1] for l in l_de_ce[50:60]]
type(ll), len(ll)

In [ ]:
#  https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html
SC = [[len(c), DG.subgraph(c).copy()] for c in nx.strongly_connected_components(DG)]
len(SC)

In [ ]:
sadg = [[s[0], s[1]] for s in SC  if s[0] == 3]
sadg

In [ ]:
## https://networkx.org/documentation/stable/reference/drawing.html
plt.figure(figsize=(15,7))

pos = nx.shell_layout(sadg[0][1])
#pos = nx.kamada_kawai_layout(a[0][1])

## Attention pas plus d'une cinquantaine de noeuds
nx.draw_networkx(sadg[0][1], pos)

## Shortest path

In [ ]:
sp = dict(nx.all_pairs_shortest_path(DG)) # , cutoff=10
len(sp)

In [ ]:
list(sp.items())[3:5]

In [ ]:
test_l = list(sp.items())[3:5]

In [ ]:
all_items = list(sp.items())
all_paths = []
for i in all_items: # test_l:
    for k,v in i[1].items():
        if i[0] != k and len(v) > 2:
            all_paths.append([i[0], k, len(v), v])

print(all_paths[:3])

In [ ]:
df_all_paths = pd.DataFrame(all_paths, columns = ['tail', 'head', 'len_path', 'list_path'])
df_all_paths.head()

In [ ]:
distrib_len_path = df_all_paths.groupby('len_path').size()
distrib_len_path

In [ ]:
df_all_paths[df_all_paths['len_path'].isin([12])]

In [ ]:
df_all_paths[df_all_paths['len_path'].isin([12,11,10,9,8])]['list_path']

In [ ]:
long_paths_nodes = []
[long_paths_nodes.extend(p) for p in df_all_paths[df_all_paths['len_path'].isin([12,11])]['list_path']] # ,10,9,8
len(list(set(long_paths_nodes)))

In [ ]:
long_paths_nodes[:3]

## Créer nouveau graphe

In [ ]:
LPN = DG.subgraph(long_paths_nodes).copy()
type(LPN)

In [ ]:
### Propriétés du graphe
LPN.is_multigraph(), LPN.is_directed(), LPN.number_of_nodes(), LPN.number_of_edges(), \
nx.density(LPN), nx.is_strongly_connected(LPN), nx.is_weakly_connected(LPN)

In [ ]:
nx.number_strongly_connected_components(LPN), nx.number_weakly_connected_components(LPN)

In [ ]:
## https://networkx.org/documentation/stable/reference/drawing.html
plt.figure(figsize=(50,100))

ax = plt.subplot(111)
ax.set_title('Graph - PLT', fontsize=10)




# pos = nx.spiral_layout(LPN)
pos = nx.kamada_kawai_layout(LPN)
# pos = nx.kamada_kawai_layout(LPN)

## Attention pas plus d'une cinquantaine de noeuds
nx.draw_networkx(LPN, pos)
plt.tight_layout()
plt.savefig("graphs/LPN.png", format="PNG")
plt.show()

In [ ]:
directed = ipycytoscape.CytoscapeWidget()
directed.graph.add_graph_from_networkx(LPN, directed=True)  # LPN
directed

In [ ]:
nx.write_pajek(DG, "graphs/DG.net")

In [ ]:
nx.write_pajek(LPN, "graphs/LPN.net")